# 利用 xpath 拆解法 - 自由時報關鍵字查詢為例, 完整內文

In [6]:
#coding:utf-8
#65001
import urllib.request
import requests
import json
import codecs
import sys
import argparse as ap
import time
import datetime
from urllib.parse import quote
from lxml import etree

#python main.py 八仙塵爆 2015-06-27 2015-08-24 1
#def argParse():
#    parser=ap.ArgumentParser(description='Liberty Time Net Crawler')
#    parser.add_argument("keyword", help="Serch Keyword")
#    parser.add_argument("start_date", help="Start (2017-01-01)")
#    parser.add_argument("end_date", help="End (2017-02-02)" )
#    parser.add_argument("pages", help="Pages")
#    return parser.parse_args()

#args=argParse()
#keyword = args.keyword
#start_date = args.start_date
#end_date = args.end_date
#pages = args.pages

keyword = '川普'
start_date = '2019-01-05'
end_date = '2019-01-05'
pages = '1'
rs = requests.session()

def start_requests():
    if( len(start_date.split("-"))==3 and len(end_date.split("-"))==3) :
        SYear = start_date.split("-")[0]
        SMonth = start_date.split("-")[1]
        SDay = start_date.split("-")[2]
        EYear = end_date.split("-")[0]
        EMonth = end_date.split("-")[1]
        EDay = end_date.split("-")[2]
        urls = []
        for i in range(1,int(pages)+1):
            str_idx = ''+('%s' % i)
            api = "https://news.ltn.com.tw/search?keyword={}&conditions=and&start_time={}&end_time={}&page={}".format(quote(keyword), start_date, end_date, str_idx)
            urls.append(api)

        for url in urls:
            print (url)
            parseLtnNews(url)
            time.sleep(0.2)
    else:
        print ("Data format error.")


def request_uri(uri):
    header = {"User-Agent": 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:51.0)'}
    res = rs.get(uri, headers=header)
    html_data =  res.text
    return html_data


def parseLtnNews(uri):
    title = []
    link = []
    body = []
    postdate = []
    html_data =  request_uri(uri)
    selector = etree.HTML(html_data)
    newslist = selector.xpath('//*[@class="searchlist boxTitle"]/li')
    
    for i in range(len(newslist)):
        strTitle = ''
        strUrl = ''
        strBody = ''
        strDate = ''
        str_idx = str(i+1)
        str_xpath = '//*[@class="searchlist boxTitle"]/li['+str_idx+']/a//text()'
        titleList = selector.xpath(str_xpath)
        strTitle = " ".join(titleList)
        str_xpath = '//*[@class="searchlist boxTitle"]/li['+str_idx+']/a//@href'
        urlList = selector.xpath(str_xpath)[0]
        strUrl = ''.join(urlList)
        try:
            html_data2 = request_uri(strUrl)
            print(strUrl)
            selector2 = etree.HTML(html_data2)
            tmp_list = selector2.xpath('//*[@class="text boxTitle boxText"]/p//text()')
            strBody = ''.join(tmp_list).replace('\n','').replace('\r','').strip().split('$(')[0]
        except Exception as e:
            print('Exception:'+str(e))
            pass
        if (len(strBody)<2) and (len(html_data2))>10:
            tmp_list = selector2.xpath('//*[@itemprop="articleBody"]//text()')
            strBody = ''.join(tmp_list).replace('\n','').replace('\r','').strip().split('$(')[0].split('不用抽')[0]
            pass

        str_xpath = '//*[@class="searchlist boxTitle"]/li['+str_idx+']/span//text()'
        dateList = selector.xpath(str_xpath)
        strDate = ''.join(dateList).replace("&nbsp;","")[:10]
        if len(strTitle)>1:
            items.append({
                "title": strTitle,
                "link":urlList,
                "body":strBody,
                "postdate":strDate,
                #"updatetime":datetime.datetime.now(),  # MongoDB
                "updatetime":datetime.datetime.now().strftime('%Y-%m-%d')
                })


if __name__ == '__main__':
    items = []
    start_requests();
    row_json = json.dumps(items, ensure_ascii=False)
    file = codecs.open(urllib.parse.unquote(keyword)+'.json', 'w', encoding='utf-8')
    #file = codecs.open('out.json', 'w', encoding='utf-8')
    file.write(row_json)
    file.close()
    print("Done")

http://news.ltn.com.tw/search?keyword=川普&conditions=and&SYear=2019&SMonth=01&SDay=05&EYear=2019&EMonth=01&EDay=05&page=1
https://news.ltn.com.tw/news/world/breakingnews/2982281
https://news.ltn.com.tw/news/business/paper/1333121
https://news.ltn.com.tw/news/world/breakingnews/2982280
https://news.ltn.com.tw/news/world/breakingnews/2982277
https://news.ltn.com.tw/news/world/breakingnews/2982273
https://news.ltn.com.tw/news/politics/breakingnews/2981863
https://news.ltn.com.tw/news/business/breakingnews/2981891
https://news.ltn.com.tw/news/business/breakingnews/2981634
https://news.ltn.com.tw/news/world/breakingnews/2981341
https://news.ltn.com.tw/news/world/breakingnews/2981588
https://news.ltn.com.tw/news/world/breakingnews/2981425
https://news.ltn.com.tw/news/business/breakingnews/2981365
https://news.ltn.com.tw/news/world/breakingnews/2981366
https://news.ltn.com.tw/news/world/paper/1332879
https://news.ltn.com.tw/news/weeklybiz/paper/1332868
Done
